In [1]:
# Unzip the dataset
!apt install unzip
!unzip /content/gender_dataset.zip -d /content/dataset

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Archive:  /content/gender_dataset.zip
replace /content/dataset/Training/female/131422.jpg.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [2]:
# Import Libraries
import os
import requests
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from google.colab import files
from tensorflow.keras.utils import img_to_array, load_img

In [3]:
# Define paths
LOCAL_DATASET_PATH = "/content/dataset"
TRAIN_DIR = os.path.join(LOCAL_DATASET_PATH, "Training")
VALIDATION_DIR = os.path.join(LOCAL_DATASET_PATH, "Validation")

# Load the Pretrained Model from the new path
PRETRAINED_MODEL_PATH = "/content/rcmalli_vggface_tf_notop_vgg16.h5"
PRETRAINED_MODEL_URL = "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_tf_notop_vgg16.h5"  # Replace with the actual URL

In [4]:
# Check if the file exists and download if not
if not os.path.exists(PRETRAINED_MODEL_PATH):
    print("Downloading pretrained model...")
    response = requests.get(PRETRAINED_MODEL_URL, stream=True)
    with open(PRETRAINED_MODEL_PATH, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Download complete.")
else:
    print("Pretrained model file already exists.")

Pretrained model file already exists.


In [5]:
def create_gender_classifier(weights_path, input_shape=(224, 224, 3)):
    # Load the ResNet50 base model without the top layers
    base_model = tf.keras.applications.VGG16(
        include_top=False,  # Exclude the fully connected layers
        weights=None,  # Initialize without weights
        input_shape=input_shape
    )

    # Load the pretrained weights
    base_model.load_weights(weights_path)

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add Custom Classification Head
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.35)(x)
    x = layers.Dense(1, activation='sigmoid')(x)  # 1 output for binary classification

    # Final Model
    model = models.Model(inputs=base_model.input, outputs=x)
    return model

In [6]:
# Create the model with the pretrained weights
gender_model = create_gender_classifier(PRETRAINED_MODEL_PATH)

In [7]:
# Compile the Model
gender_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',  # Use binary crossentropy for binary classification
    metrics=['accuracy']
)

In [8]:
gender_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       1,605,696 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 16,320,449 (62.26 MB)

 Trainable params: 1,605,761 (6.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [9]:
# Prepare training and validation data generators without data augmentation
train_gen = ImageDataGenerator(rescale=1./255)
validation_gen = ImageDataGenerator(rescale=1./255)

# Prepare training and validation data generators
train_generator = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',  # Use binary class mode
)

validation_generator = validation_gen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',  # Use binary class mode
)

Found 47009 images belonging to 2 classes.
Found 11649 images belonging to 2 classes.


In [10]:
# Train the Model
history = gender_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=1,
)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


735/735 ━━━━━━━━━━━━━━━━━━━━ 292s 359ms/step - accuracy: 0.9549 - loss: 0.1283 - val_accuracy: 0.9737 - val_loss: 0.0803


In [11]:
# Save the Model
SAVE_MODEL_PATH = "/content/gender_classifier_model.h5"
gender_model.save(SAVE_MODEL_PATH)

In [20]:
# # Plot Training History
# def plot_training_history(history):
#     plt.figure(figsize=(12, 6))

#     plt.subplot(1, 2, 1)
#     plt.plot(history.history.get('accuracy', []), label='Training Accuracy')
#     plt.plot(history.history.get('val_accuracy', []), label='Validation Accuracy')
#     plt.legend()
#     plt.title('Accuracy')

#     plt.subplot(1, 2, 2)
#     plt.plot(history.history.get('loss', []), label='Training Loss')
#     plt.plot(history.history.get('val_loss', []), label='Validation Loss')
#     plt.legend()
#     plt.title('Loss')

#     plt.show()

# plot_training_history(history)

In [13]:
# Evaluate the Model
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_directory(
    VALIDATION_DIR,  # Use validation data for evaluation
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Use binary class mode
    shuffle=False  # Important for confusion matrix
)

# Get predictions
predictions = gender_model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype("int32")  # Convert probabilities to binary predictions
true_classes = test_generator.classes


Found 11649 images belonging to 2 classes.
365/365 ━━━━━━━━━━━━━━━━━━━━ 61s 129ms/step


In [14]:
# Confusion Matrix and Classification Report
cm = confusion_matrix(true_classes, predicted_classes)
report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys())

In [19]:
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)

Confusion Matrix:
[[5773   68]
 [ 238 5570]]

Classification Report:
              precision    recall  f1-score   support

      female       0.96      0.99      0.97      5841
        male       0.99      0.96      0.97      5808

    accuracy                           0.97     11649
   macro avg       0.97      0.97      0.97     11649
weighted avg       0.97      0.97      0.97     11649



In [17]:
# Function to predict a single image
def predict_image(image_path, model, class_mapping, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image) / 255.0  # Normalize to [0, 1]
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

    predictions = model.predict(image_array)
    predicted_index = (predictions[0] > 0.5).astype("int32")  # Convert probability to binary

    return class_mapping[predicted_index[0]]

# Define class mapping
classes = {
    0: "Female",
    1: "Male"
}

# Upload an image and predict the class
uploaded = files.upload()  # Upload an image
for file_name in uploaded.keys():
    predicted_class = predict_image(file_name, gender_model, classes)
    print(f"Predicted class for '{file_name}': {predicted_class}")

Saving f8ef2c2f-17b8-4bd4-a572-7c35d7faa73a.jpg to f8ef2c2f-17b8-4bd4-a572-7c35d7faa73a.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted class for 'f8ef2c2f-17b8-4bd4-a572-7c35d7faa73a.jpg': Male
